#Implement Data Quality in Notebooks by using Soda

1. Install required dev tools
2. Install Libraries for 
  1. [Soda](https://www.soda.io/) -> Data quality framework
  2. [GE](https://greatexpectations.io/) - Data quality framework
  3. [Influxdb](https://www.influxdata.com/) - Time series database to publish and visualize measurements.
3. Read data from a sample csv and create Dataframe.
4. Create methods to return Soda scan results to Dataframe. (For easier analysis)
5. Define Soda scan yml and execute on Dataframe.
  1. The results are also published to a free trial version of Soda Cloud.
6. Display the Scan results from the Dataframe.

##### TODO:
1. Explore GreatExpectations for Data quality
2. Explore publishing data to InfluxDB to replicate the dashboards in Soda Cloud.

In [0]:
%run ./SODA_CLUSTER_SETUP

You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.
  Created wheel for thrift: filename=thrift-0.15.0-cp38-cp38-linux_x86_64.whl size=399949 sha256=4dca8920eb8415562a34b7d3dae6727847c3e637cd9603bc395422249b81aff7
  Stored in directory: /root/.cache/pip/wheels/4c/b5/5b/10af165d7e0895afdfe25ad487422ae8ada6ea422b0dc444ab
Successfully built thrift
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.
ii  gcc                            4:9.3.0-1ubuntu2                  amd64        GNU C compiler
ii  gcc-10-base:amd64              10.3.0-1ubuntu1~20.04             amd64        GCC, the GNU Compiler Collection (base package)
ii  gcc-7-base:amd64               7.5.0-6ubuntu2                    amd64        GCC, the GNU Compiler Collection (base package)
ii  gcc-8-base:amd64               8.4.0-3ubuntu2                    amd64        GCC, the GNU Compiler Collection (base package)


Python interpreter will be restarted.
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853646 sha256=b4842efe4dbe9c0e336c983e6b73ff97080546873652fbab129f993a22a05813
  Stored in directory: /root/.cache/pip/wheels/58/94/83/915c9059e4b038e2d43a6058f307fe1c3e8536e5745f3b23b7
  Created wheel for PyHive: filename=PyHive-0.6.4-py3-none-any.whl size=51493 sha256=3bbff8ab2fda9b019a624def132ee4cdbc2744af73851f389ad4a6875ce92121
  Stored in directory: /root/.cache/pip/wheels/84/f1/d7/6f7af9915dcb9abad04a80249755d01819315f77a3bbc976ba
  Created wheel for pyodbc: filename=pyodbc-4.0.32-cp38-cp38-linux_x86_64.whl size=359036 sha256=b0b73d3798780cbe25c92b33236ca7381e409893b3d0241664edaa859d789b7c
  Stored in directory: /root/.cache/pip/wheels/68/de/de/65a129482924e96fb701c51488b907953acf25fe623bb177b3
  Created wheel for sasl: filename=sasl-0.3.1-cp38-cp38-linux_x86_64.whl size=292635 sha256=54cf16d6d6cb0448e1ef544a0ce8887990d403e6ff9373b1c34e61340df1c419
  Stored in 

In [0]:
%run ./SODA_PROPERTIES

In [0]:
#Column names with a space caused the Soda scan to fail So i renamed the column names with space. 

from pyspark.sql.types import StructField, StructType, StringType,IntegerType
from pyspark.sql.functions import length

# File location and type
file_location = "/FileStore/tables/all_india_PO_list_without_APS_offices_ver2_lat_long.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
#df = spark.read.format(file_type).schema(custom_schema).option("header", first_row_is_header).option("sep", delimiter).load(file_location)

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

df = df.withColumnRenamed("Related Suboffice","RelatedSuboffice").withColumnRenamed("Related Headoffice","RelatedHeadoffice")


#df.where("Deliverystatus").show()
df = df.filter(df.Deliverystatus == "Delivery")

In [0]:
# Create a view or table

temp_table_name = "allindiapo"

df.createOrReplaceTempView(temp_table_name)

In [0]:
%sql

/* Query the created temp table in a SQL cell */

select * from allindiapo

officename,pincode,officeType,Deliverystatus,divisionname,regionname,circlename,Taluk,Districtname,statename,Telephone,RelatedSuboffice,RelatedHeadoffice,longitude,latitude
Achalapur B.O,504273,B.O,Delivery,Adilabad,Hyderabad,Andhra Pradesh,Asifabad,Adilabad,TELANGANA,NA,Rechini S.O,Mancherial H.O,NA,NA
Ada B.O,504293,B.O,Delivery,Adilabad,Hyderabad,Andhra Pradesh,Asifabad,Adilabad,TELANGANA,NA,Asifabad S.O,Mancherial H.O,NA,NA
Adegaon B.O,504307,B.O,Delivery,Adilabad,Hyderabad,Andhra Pradesh,Boath,Adilabad,TELANGANA,NA,Echoda S.O,Adilabad H.O,NA,NA
Adilabad H.O,504001,H.O,Delivery,Adilabad,Hyderabad,Andhra Pradesh,Adilabad,Adilabad,TELANGANA,08732-226738,NA,NA,NA,NA
Ainam B.O,504273,B.O,Delivery,Adilabad,Hyderabad,Andhra Pradesh,Asifabad,Adilabad,TELANGANA,NA,Rechini S.O,Mancherial H.O,NA,NA
Aknepalli B.O,504251,B.O,Delivery,Adilabad,Hyderabad,Andhra Pradesh,Luxettipet,Adilabad,TELANGANA,NA,Bellampalli S.O,Mancherial H.O,NA,NA
Akoli B.O,504309,B.O,Delivery,Adilabad,Hyderabad,Andhra Pradesh,Adilabad,Adilabad,TELANGANA,NA,Gumma S.O (Adilabad),Adilabad H.O,NA,NA
Alegaon B.O,504201,B.O,Delivery,Adilabad,Hyderabad,Andhra Pradesh,Chennur,Adilabad,TELANGANA,NA,Chinnoor S.O,Mancherial H.O,NA,NA
Aloor B.O,504110,B.O,Delivery,Adilabad,Hyderabad,Andhra Pradesh,Nirmal,Adilabad,TELANGANA,NA,Chincholi (B) S.O,Adilabad H.O,NA,NA
Ambaripet B.O,504202,B.O,Delivery,Adilabad,Hyderabad,Andhra Pradesh,Khanapur,Adilabad,TELANGANA,NA,Peddur S.O,Adilabad H.O,NA,NA


In [0]:
from pyspark.sql import DataFrame, SparkSession
from sodaspark import scan
import pyodbc

#Enable connection to Soda Cloud
import os
from sodasql.soda_server_client.soda_server_client import SodaServerClient
soda_server_client = SodaServerClient(
  host=SODA_HOST,
  api_key_id=SODA_API_KEY_ID,
  api_key_secret =SODA_API_KEY_SECRET
    )
#Define the soda scan yml file
#Lot of documentation available at https://docs.soda.io/soda-sql/sql_metrics.html#metric-groups-and-dependencies
scan_definition = (
  """
    table_name: podata
    metrics:
    - row_count
    - max_length
    - unique_count
    - distinct
    - duplicate_count
    metric_groups:
    - length
    - profiling
    samples:
      table_limit: 50
    tests:
    - row_count > 0
    columns:
      Deliverystatus:
        tests:
        - distinct == 2
      divisionname:
        tests:
        - maxs
    sql_metrics:
    - sql: SELECT statename, count(pincode) as total_pincode FROM podata GROUP BY statename
      group_fields:
      - statename
      tests:
      - total_pincode > 20
    - sql: SELECT count(statename) as telangana_count FROM podata WHERE statename = 'TELANGANA'
      tests:
      - telangana_count > 30
    """
)



In [0]:
#Execute the scan based on tests defined
scan_result = scan.execute(scan_definition, df, soda_server_client=soda_server_client, as_frames=True)
measurements_df = scan_result[0]
testResults_df = scan_result[1]
errors_df = scan_result[2]



Soda Cloud error: Could not start scan: HTTPSConnectionPool(host="('cloud.soda.io',)", port=443): Max retries exceeded with url: /api/command (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f0b43954df0>: Failed to establish a new connection: [Errno -2] Name or service not known'))
Skipping subsequent Soda Cloud communication but continuing the scan


In [0]:
#Convert them to Dataframe and maybe persist in adatabase for visualization.
#If you have signed up for Soda Cloud - That takes care of visualization/alerts and incidents.
display(measurements_df)
display(testResults_df)
display(testResults_df.select("test.*", "passed","skipped","values","error"))


metric,column_name,value,group_values
schema,null,"[{logicalType=text, nullable=true, semanticType=text, dataType=string, name=officename, type=string}, {logicalType=number, nullable=true, semanticType=number, dataType=int, name=pincode, type=int}, {logicalType=text, nullable=true, semanticType=text, dataType=string, name=officeType, type=string}, {logicalType=text, nullable=true, semanticType=text, dataType=string, name=Deliverystatus, type=string}, {logicalType=text, nullable=true, semanticType=text, dataType=string, name=divisionname, type=string}, {logicalType=text, nullable=true, semanticType=text, dataType=string, name=regionname, type=string}, {logicalType=text, nullable=true, semanticType=text, dataType=string, name=circlename, type=string}, {logicalType=text, nullable=true, semanticType=text, dataType=string, name=Taluk, type=string}, {logicalType=text, nullable=true, semanticType=text, dataType=string, name=Districtname, type=string}, {logicalType=text, nullable=true, semanticType=text, dataType=string, name=statename, type=string}, {logicalType=text, nullable=true, semanticType=text, dataType=string, name=Telephone, type=string}, {logicalType=text, nullable=true, semanticType=text, dataType=string, name=RelatedSuboffice, type=string}, {logicalType=text, nullable=true, semanticType=text, dataType=string, name=RelatedHeadoffice, type=string}, {logicalType=text, nullable=true, semanticType=text, dataType=string, name=longitude, type=string}, {logicalType=text, nullable=true, semanticType=text, dataType=string, name=latitude, type=string}]",null
row_count,null,145298,null
values_count,officename,145298,null
valid_count,officename,145298,null
avg_length,officename,13.189961320871587,null
min_length,officename,5,null
max_length,officename,50,null
values_count,pincode,145298,null
valid_count,pincode,145298,null
min,pincode,110001,null


test,passed,skipped,values,error,group_values
"List({""sql_metric_index"":0,""expression"":""total_pincode > 20""}, sqlmetric(0) test(total_pincode > 20), total_pincode > 20, List(total_pincode), null, soda-sql)",true,false,"Map(expression_result -> 3060, total_pincode -> 3060)",null,Map(statename -> CHATTISGARH)
"List({""sql_metric_index"":0,""expression"":""total_pincode > 20""}, sqlmetric(0) test(total_pincode > 20), total_pincode > 20, List(total_pincode), null, soda-sql)",true,false,"Map(expression_result -> 8802, total_pincode -> 8802)",null,Map(statename -> BIHAR)
"List({""sql_metric_index"":0,""expression"":""total_pincode > 20""}, sqlmetric(0) test(total_pincode > 20), total_pincode > 20, List(total_pincode), null, soda-sql)",true,false,"Map(expression_result -> 76, total_pincode -> 76)",null,Map(statename -> PONDICHERRY)
"List({""sql_metric_index"":0,""expression"":""total_pincode > 20""}, sqlmetric(0) test(total_pincode > 20), total_pincode > 20, List(total_pincode), null, soda-sql)",true,false,"Map(expression_result -> 5555, total_pincode -> 5555)",null,Map(statename -> TELANGANA)
"List({""sql_metric_index"":0,""expression"":""total_pincode > 20""}, sqlmetric(0) test(total_pincode > 20), total_pincode > 20, List(total_pincode), null, soda-sql)",true,false,"Map(expression_result -> 9872, total_pincode -> 9872)",null,Map(statename -> ANDHRA PRADESH)
"List({""sql_metric_index"":0,""expression"":""total_pincode > 20""}, sqlmetric(0) test(total_pincode > 20), total_pincode > 20, List(total_pincode), null, soda-sql)",true,false,"Map(expression_result -> 3729, total_pincode -> 3729)",null,Map(statename -> ASSAM)
"List({""sql_metric_index"":0,""expression"":""total_pincode > 20""}, sqlmetric(0) test(total_pincode > 20), total_pincode > 20, List(total_pincode), null, soda-sql)",false,false,"Map(expression_result -> 9, total_pincode -> 9)",null,Map(statename -> NULL)
"List({""sql_metric_index"":0,""expression"":""total_pincode > 20""}, sqlmetric(0) test(total_pincode > 20), total_pincode > 20, List(total_pincode), null, soda-sql)",false,false,"Map(expression_result -> 19, total_pincode -> 19)",null,Map(statename -> DAMAN & DIU)
"List({""sql_metric_index"":0,""expression"":""total_pincode > 20""}, sqlmetric(0) test(total_pincode > 20), total_pincode > 20, List(total_pincode), null, soda-sql)",true,false,"Map(expression_result -> 162, total_pincode -> 162)",null,Map(statename -> DELHI)
"List({""sql_metric_index"":0,""expression"":""total_pincode > 20""}, sqlmetric(0) test(total_pincode > 20), total_pincode > 20, List(total_pincode), null, soda-sql)",true,false,"Map(expression_result -> 2737, total_pincode -> 2737)",null,Map(statename -> HIMACHAL PRADESH)


id,title,expression,metrics,column,source,passed,skipped,values,error
"{""sql_metric_index"":0,""expression"":""total_pincode > 20""}",sqlmetric(0) test(total_pincode > 20),total_pincode > 20,List(total_pincode),null,soda-sql,true,false,"Map(expression_result -> 3060, total_pincode -> 3060)",null
"{""sql_metric_index"":0,""expression"":""total_pincode > 20""}",sqlmetric(0) test(total_pincode > 20),total_pincode > 20,List(total_pincode),null,soda-sql,true,false,"Map(expression_result -> 8802, total_pincode -> 8802)",null
"{""sql_metric_index"":0,""expression"":""total_pincode > 20""}",sqlmetric(0) test(total_pincode > 20),total_pincode > 20,List(total_pincode),null,soda-sql,true,false,"Map(expression_result -> 76, total_pincode -> 76)",null
"{""sql_metric_index"":0,""expression"":""total_pincode > 20""}",sqlmetric(0) test(total_pincode > 20),total_pincode > 20,List(total_pincode),null,soda-sql,true,false,"Map(expression_result -> 5555, total_pincode -> 5555)",null
"{""sql_metric_index"":0,""expression"":""total_pincode > 20""}",sqlmetric(0) test(total_pincode > 20),total_pincode > 20,List(total_pincode),null,soda-sql,true,false,"Map(expression_result -> 9872, total_pincode -> 9872)",null
"{""sql_metric_index"":0,""expression"":""total_pincode > 20""}",sqlmetric(0) test(total_pincode > 20),total_pincode > 20,List(total_pincode),null,soda-sql,true,false,"Map(expression_result -> 3729, total_pincode -> 3729)",null
"{""sql_metric_index"":0,""expression"":""total_pincode > 20""}",sqlmetric(0) test(total_pincode > 20),total_pincode > 20,List(total_pincode),null,soda-sql,false,false,"Map(expression_result -> 9, total_pincode -> 9)",null
"{""sql_metric_index"":0,""expression"":""total_pincode > 20""}",sqlmetric(0) test(total_pincode > 20),total_pincode > 20,List(total_pincode),null,soda-sql,false,false,"Map(expression_result -> 19, total_pincode -> 19)",null
"{""sql_metric_index"":0,""expression"":""total_pincode > 20""}",sqlmetric(0) test(total_pincode > 20),total_pincode > 20,List(total_pincode),null,soda-sql,true,false,"Map(expression_result -> 162, total_pincode -> 162)",null
"{""sql_metric_index"":0,""expression"":""total_pincode > 20""}",sqlmetric(0) test(total_pincode > 20),total_pincode > 20,List(total_pincode),null,soda-sql,true,false,"Map(expression_result -> 2737, total_pincode -> 2737)",null


In [0]:
from datetime import datetime
import influxdb_client
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS

# You can generate an API token from the "API Tokens Tab" in the UI
token = INFLUX_TOKEN
org = INFLUX_ORG
bucket = INFLUX_BUCKET
url= INFLUX_URL


soda_measurements = scan_result[0].filter(scan_result[0].metric == "unique_count")
display(soda_measurements)

client = influxdb_client.InfluxDBClient(
    url=url,
    token=token,
    org=org
)

write_api = client.write_api(write_options=SYNCHRONOUS)

# soda_measurements = scan_result[0].collect()
for row in soda_measurements.collect():
  p = influxdb_client.Point(row["metric"]).tag("column_name", row["column_name"]).field("value", int(row["value"]))
  write_api.write(bucket=bucket, org=org, record=p)

client.close()

metric,column_name,value,group_values
unique_count,officename,110893,null
unique_count,pincode,3332,null
unique_count,officeType,0,null
unique_count,Deliverystatus,0,null
unique_count,divisionname,13,null
unique_count,regionname,0,null
unique_count,circlename,0,null
unique_count,Taluk,3682,null
unique_count,Districtname,4,null
unique_count,statename,0,null
